# GenAI Prompt Engineering Examples

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [11]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [12]:
REGION = 'us-central1'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

packages:

In [13]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import io
import base64
import asyncio
import requests
import IPython
import datetime, time

In [14]:
aiplatform.__version__

'1.67.1'

clients:

In [15]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

---
## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with.  There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:
- [vertexai.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    - [vertexai.preview.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):
- [vertexai.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    - [vertexai.preview.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):
- [vertexai.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    - [vertexai.preview.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)


In [16]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-002")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision-001")
gemini15_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")
gemini_flash = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

---
## Gemini Models

[Reference: Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)

### Text Prompt

In [36]:
goal_prompt = "Create a fantasy football team name"
goal_detail_prompt = "The name should be creative and funny, and leverage football references and a current player's name"
prompt = """
Goal: {}

Additional information: {}

You will use a “playoff” method to achieve the goal.

Follow these steps carefully:

Step 1: Generate Options 
Create a list of 32 distinct options. Each option should reflect the topic of the goal. Be creative, but make sure the options are clear, catchy, and relevant to the theme.

Step 2: The Playoff Rounds 
Round of 32: Pair the 32 options into 16 matchups (pairs). Compare each pair and choose a winner based on: How memorable and catchy the option is. How well the option reflects the goal’s theme. Other relevant qualities, such as creativity and uniqueness.

Round of 16: Take the 16 winners from the first round and pair them into 8 matchups. Compare each pair, select a winner, and explain your reasoning.

Quarterfinals: Take the 8 remaining options and form 4 matchups. Again, compare each pair, choose winners, and provide reasoning.

Semifinals: Compare the 4 remaining options in 2 matchups. Select the best option from each pair with reasoning.

Final Round: Compare the final 2 options. Choose the overall best option based on how well it represents the goal, its catchiness, and other factors. Provide a detailed explanation for why this option stands out as the best.

Step 3: The Final Output At the end of this playoff process, present: The complete list of the original 32 options, ranked from best to worst, in order. The final chosen option (the top-ranked option). A detailed explanation for why the final option was selected as the best out of all 32.
""".format(goal_prompt, goal_detail_prompt)

#print(prompt)

In [37]:
#response = gemini15_multimodal.generate_content(prompt)
#response

In [38]:
#print(response.text)

In [39]:
#IPython.display.Markdown(response.text)

### Streaming Response

In [42]:
for response in gemini15_multimodal.generate_content(prompt, stream = True):
    print(response.text)

**
Step 1: Generate Options**

Here are 32 fantasy football team
 names incorporating a current player (we'll use Patrick Mahomes for this example)
 and football references:

1. Mahomes and the Misfits
2. The Patrick Mahomes-ters
3. Mahomes' Magnificent Seven
4. Air Mahomes

5. The Gridiron G.O.A.T.s (Mahomes Edition)
6. Mahomes' Magic
7. Touchdown
 Mahomes & Co.
8. The Mahomes Mob
9. Patty Mahomes' Passing Parade
10.  Mahomes' Merry Men
11.  The Kingdom Comebacks (referencing Mahomes' team)
12.
  Mahomes and the Hail Marys
13.  Mahomes' Money Makers
14.  The Mahomes Maniacs
15.  Mahomes' Midnight Riders
16.  The Gridiron Gods (Mah
omes' Squad)
17.  Mahomes' Marvels
18.  The Patrick Mahomes Experience
19.  Mahomes and the Maulers
20.  The Mahomes Dynasty
21.  Mahomes' Magnificent Militia
22.  The Chiefs' Chosen One
2
3.  Mahomes' Masterminds
24.  The Mahomes' Matrix
25.  Mahomes and the Muscle
26.  Mahomes' Missile Command
27.  The Field Goal Fanatics (featuring Mahomes)
28.  Mahomes' 